# Tableau Project (Criterio 1: df_capability)

In [4]:
#Tras iniciar servidor de DB mongo, importo módulo 'pymongo'.
import pymongo
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize
#Asigno variable al cliente de DB y a la DB.
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies
cursor = db.companies.find()
data = list(cursor)
df = pd.DataFrame(data)

In [5]:
col_float = ['deadpooled_day','deadpooled_month','deadpooled_year','founded_day','founded_month',
             'founded_year','number_of_employees']
col_obj = ['_id','acquisition','acquisitions','alias_list','blog_feed_url','blog_url','category_code',
           'competitions','created_at','crunchbase_url','deadpooled_url','description','email_address',
        'external_links','funding_rounds','homepage_url','image','investments','ipo','milestones', 
           'name','offices','overview','partners','permalink','phone_number','products','providerships',
        'relationships','screenshots','tag_list','total_money_raised','twitter_username','updated_at',
           'video_embeds']
col_drop1 = ['deadpooled_day','deadpooled_month','founded_day','founded_month','alias_list','blog_feed_url',
             'blog_url','created_at','crunchbase_url','deadpooled_url','email_address','external_links',
            'image','permalink','phone_number','screenshots','tag_list','updated_at','video_embeds']
col_drop2 = ['acquisition','acquisitions','competitions','description','funding_rounds','homepage_url',
             'investments','ipo','milestones','partners','products','providerships','relationships',
             'twitter_username','overview',]
col_ok = ['deadpooled_year','founded_year','number_of_employees','_id','category_code','name','offices',
          'total_money_raised']

In [6]:
#Elimino columnas con valores que no me interesan
df_drop1 = df.drop(col_drop1, axis=1)
df_drop2 = df_drop1.drop(col_drop2, axis=1)
#Elijo solo las empresas con deadpooled_year == null
df_deadnull = df_drop2[df_drop2['deadpooled_year'].isnull()]
#Elimino filas con valores nulos de las columnas que me interesan
df_nulls = df_deadnull.dropna(subset=['founded_year','number_of_employees','name','offices',
          'total_money_raised'])
display(len(df_nulls))

7934

In [7]:
#Data frames por tipo de empresas y elimino las columnas _id y deadpooled_year 
col_cat = ['games_video']
df_cat = df_nulls[df_nulls['category_code'].isin(col_cat)]\
.drop(['_id','deadpooled_year'], axis=1)
print(len(df_cat))

525


In [8]:
df_regout = df_cat.copy()
df_regout = df_regout[df_regout.number_of_employees != 0]
df_regout = df_regout[df_regout.total_money_raised != 0]
df_regout = df_regout[df_regout['offices'].map(len) > 0]
print(len(df_regout))

398


In [9]:
#Defino funciones para la modificación de los datos de las columnas a analizar.

#Transforma un dato tipo float a uno tipo int.
def float_to_int(flt):
    integer = int(flt)
    return integer

#Transforma un dato tipo str en int devolviendo solo el número.
def str_to_int(strg):
    string = re.findall('\d+', strg )
    integer = int(string[0])*1000
    return integer

#Transforma los datos de ubicación de las oficinas en formato GeoJson y selecciona solo la primera ubicación
def nested_to_list(dicts):
    lat_lon = [{"type":"Point","coordinates":[l['longitude'],l['latitude']]} for l in dicts]
    lst = lat_lon[0]
    long = lst['coordinates'][0]
    if long != None:
        return lst
    else:
        return 0